In [8]:
import tensorflow as tf
import numpy as np
import random

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777) 

mnist = input_data.read_data_sets("../data/", one_hot=True)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [10]:
# parameters
learning_rate = 0.001
training_epochs = 50
batch_size = 100

X = tf.placeholder(tf.float32, [None, 784]) # (?, 784)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10]) # (? 10)

# Layer1
# (?, 28, 28, 1) => (?, 24, 24, 6)
conv1 = tf.layers.conv2d(inputs = X_img, filters  = 6, kernel_size = [5, 5], strides = (1, 1), activation = tf.nn.relu)
# (?, 24, 24, 6) => (?, 12, 12, 6)
pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2, 2], strides = (2, 2))

# Layer2
# (?, 12, 12, 6) => (?, 8, 8, 16)
conv2 = tf.layers.conv2d(inputs = pool1, filters = 16, kernel_size = [5, 5], strides = (1, 1), activation = tf.nn.relu)
# (?, 8, 8, 16) => (?, 4, 4, 16)
pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2, 2], strides = (2, 2))
# (?, 4, 4, 16) => (?, 4*4*16 = 256)
flat = tf.contrib.layers.flatten(pool2)

# Layer3
# (?, 256) => (?, int(256*0.7) = 76)
dense3 = tf.layers.dense(inputs = flat, units = 76, activation = tf.nn.relu)

# Layer4
# (?, 76) => (?, int(76 * 0.7) = 53)
dense4 = tf.layers.dense(inputs = dense3, units = 53, activation = tf.nn.relu)

# Layer5
# (?, 53) => (?, 10)
logits = tf.layers.dense(inputs = dense4, units = 10)

# define loss function
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels = Y)) # (?, 10)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train model
for epoch in range(training_epochs):
    avg_loss = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        feed_dict = {X : batch_x, Y : batch_y}
        l, _ = sess.run([cost, optimizer], feed_dict= feed_dict)
        avg_loss += l / total_batch
    
    print('Epoch : ', '%04d' % (epoch + 1), 'loss = ', avg_loss)

print('Learning finished')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, axis=1), tf.argmax(Y,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy : ', sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print('Label : ', sess.run(tf.argmax(mnist.test.labels[[r]], axis=1)))
print('Prediction : ', sess.run(tf.argmax(logits, 1), feed_dict={X : mnist.test.images[[r]]}))



Epoch :  0001 loss =  0.101067267008
Epoch :  0002 loss =  0.0235231034889
Epoch :  0003 loss =  0.0164525665042
Epoch :  0004 loss =  0.0131860902844
Epoch :  0005 loss =  0.0108612282216
Epoch :  0006 loss =  0.00951154066758
Epoch :  0007 loss =  0.00826710534705
Epoch :  0008 loss =  0.00726208202038
Epoch :  0009 loss =  0.00638514433977
Epoch :  0010 loss =  0.00551450972371
Epoch :  0011 loss =  0.00491723186813
Epoch :  0012 loss =  0.00437714987424
Epoch :  0013 loss =  0.00414507759187
Epoch :  0014 loss =  0.00363443252492
Epoch :  0015 loss =  0.00306440296716
Epoch :  0016 loss =  0.00300554767531
Epoch :  0017 loss =  0.00263949014944
Epoch :  0018 loss =  0.00231919124255
Epoch :  0019 loss =  0.00218346544799
Epoch :  0020 loss =  0.00203336729004
Epoch :  0021 loss =  0.00186673287706
Epoch :  0022 loss =  0.00191481531448
Epoch :  0023 loss =  0.00138022092175
Epoch :  0024 loss =  0.00151137551517
Epoch :  0025 loss =  0.00152744567412
Epoch :  0026 loss =  0.0011386